In [1]:
#This code is for adaptive GPU usage
import keras.backend as K
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

Using TensorFlow backend.


In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import nltk
import random
import numpy as np
import pandas as pd
import keras.layers as L
import keras.models as M
from keras.optimizers import Adam, Adamax, Nadam, Adagrad
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

Using TensorFlow backend.


In [26]:
d = pd.read_csv("All_Features_Sorted.csv",sep=',')

In [27]:
lid = d[d['label']==1]['labelID'].tolist()

In [28]:
p = [([i]*10) for i in lid]

In [29]:
p = np.array(p).ravel()

In [30]:
d['correct_labelID'] = p

In [31]:
d.to_csv("All_Features_Sorted.csv",sep=',',index=None)

# Data Loading

In [3]:
raw_data = pd.read_csv("All_TrainFeatures.csv",sep=',')

In [4]:
features = ['Cosine','Jaccard','BM25','Tfidf','Rouge P','Rouge R','BLEU 1','BLEU 2','DSSM','CDSSM',
                   'q_word_count','q_char_count','q_avg_word','r_word_count','r_char_count','r_avg_word']

split = int(10*((raw_data.shape[0]*0.7)//10))
print(split)

3669230


In [5]:
y = raw_data['label']
x = raw_data[features]

x_test = x.iloc[split:]
y_test = y.iloc[split:]

In [6]:
x_test.shape,y_test.shape

((1572540, 16), (1572540,))

In [7]:
d = raw_data.iloc[:split]
d.shape

(3669230, 18)

In [8]:
tr_data = d[d['label'] == 1].reset_index(drop=True)
fl_data = d[d['label'] == 0].reset_index(drop=True)

In [9]:
tr_data.shape, fl_data.shape

((366923, 18), (3302307, 18))

In [10]:
tr_ind = random.sample(range(0,tr_data.shape[0]),tr_data.shape[0])
fl_ind = random.sample(range(0,fl_data.shape[0]),tr_data.shape[0]*2)

In [15]:
train = pd.concat([tr_data.iloc[tr_ind],fl_data.iloc[fl_ind]],axis=0).sample(frac=1).reset_index( drop = True )

x_train = train[features]

y_train = train['label']

In [16]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1100769, 16), (1100769,), (1572540, 16), (1572540,))

# Tree Based Classifier Models

In [112]:
# hyperparams = {
#     "trees":{
#         "criterions" : ["gini","entropy"],
#         "max_features": ["log2","sqrt"], 
#         "max_depth":[7],
#         "n_estimators": [50,120,200]}, 
#     "gbc":{
#         "loss" : ["deviance","exponential"],
#         "learning_rate" : [0.01,0.1,1.0],
#         "n_estimators" : [40,80,120],
#         "max_depth":[7],
#         "max_features" : ["log2","sqrt"]},
#     "nn":{
#         "learning_rate" : [0.1,0.01,0.05,0.001,0.005],
#         "optim" : ["Adam","Adamax","Adagrad","Nadam"], 
#         "activation": ["tanh","sigmoid","relu","linear"]}
#     }

# results = {}
# # results['RFC'] = {}

# # for cr in hyperparams["trees"]["criterions"]:
# #     results['RFC'][str(cr)] = {}
# #     for dep in hyperparams["trees"]["max_depth"]:
# #         results['RFC'][str(cr)][str(dep)] = {}
# #         for feat in hyperparams["trees"]["max_features"]:
# #             results['RFC'][str(cr)][str(dep)][str(feat)] = {}
# #             for est in hyperparams["trees"]["n_estimators"]:
# #                 results['RFC'][str(cr)][str(dep)][str(feat)][str(est)] = {}
# #                 print("Hyperparameters used")
# #                 print("Criterion : ",cr)
# #                 print("Maximum Depth : ",dep)
# #                 print("Maximum Features : ",feat)
# #                 print("Number of Estimators : ",est)
# #                 model = RandomForestClassifier(n_estimators=est,
# #                                               criterion=cr,
# #                                               max_features=feat,
# #                                               verbose=3,
# #                                               )
                
# #                 model.fit(x_train,y_train)
# #                 y_pred = model.predict(x_test)
# #                 imp = model.feature_importances_
# #                 score = model.score(x_test,y_test)
# #                 print(score)
# #                 results['RFC'][str(cr)][str(dep)][str(feat)][str(est)]["score"] = score
                
# #                 f = pd.DataFrame(np.column_stack((features,imp)))
# #                 f.columns = ['feature','importance']
# #                 print(f.sort_values(by=['importance'],ascending=False))
# #                 results['RFC'][str(cr)][str(dep)][str(feat)][str(est)]["f_imp"] = f

# results['ETC'] = {}

# for cr in hyperparams["trees"]["criterions"]:
#     results['ETC'][str(cr)] = {}
#     for dep in hyperparams["trees"]["max_depth"]:
#         results['ETC'][str(cr)][str(dep)] = {}
#         for feat in hyperparams["trees"]["max_features"]:
#             results['ETC'][str(cr)][str(dep)][str(feat)] = {}
#             for est in hyperparams["trees"]["n_estimators"]:
#                 results['ETC'][str(cr)][str(dep)][str(feat)][str(est)] = {}
#                 print("Hyperparameters used")
#                 print("Criterion : ",cr)
#                 print("Maximum Depth : ",dep)
#                 print("Maximum Features : ",feat)
#                 print("Number of Estimators : ",est)
#                 model = ExtraTreesClassifier(n_estimators=est,
#                                               criterion=cr,
#                                               max_features=feat,
#                                               verbose=3,
#                                               warm_start=True,
#                                               )
                
#                 model.fit(x_train,y_train)
#                 y_pred = model.predict(x_test)
#                 imp = model.feature_importances_
#                 score = model.score(x_test,y_test)
#                 print(score)
#                 results['ETC'][str(cr)][str(dep)][str(feat)][str(est)]["score"] = score
                
#                 f = pd.DataFrame(np.column_stack((features,imp)))
#                 f.columns = ['feature','importance']
#                 print(f.sort_values(by=['importance'],ascending=False))
#                 results['ETC'][str(cr)][str(dep)][str(feat)][str(est)]["f_imp"] = f

# # results['GBC'] = {}

# # for ls in hyperparams["gbc"]["loss"]:
# #     results['GBC'][str(ls)] = {}
# #     for lr in hyperparams["gbc"]["learning_rate"]:
# #         results['GBC'][str(ls)][str(lr)] = {}
# #         for est in hyperparams["gbc"]["n_estimators"]:
# #             results['GBC'][str(ls)][str(lr)][str(est)] = {}
# #             for feat in hyperparams["gbc"]["max_features"]:
# #                 results['GBC'][str(ls)][str(lr)][str(est)][str(feat)] = {}
# #                 for dep in hyperparams["gbc"]["max_depth"]:
# #                     results['GBC'][str(ls)][str(lr)][str(est)][str(feat)][str(dep)] = {}
# #                     print("Hyperparameters used")
# #                     print("Loss : ",ls)
# #                     print("Learning rate : ",lr)
# #                     print("Number of Estimators : ",est)
# #                     print("Maximum Features : ",feat)
# #                     print("Maximum Depth : ",dep)
# #                     model = GradientBoostingClassifier(loss=ls,
# #                                                        learning_rate=lr,
# #                                                        n_estimators=est,
# #                                                        max_features=feat,
# #                                                        verbose=3)

# #                     model.fit(x_train,y_train)
# #                     y_pred = model.predict(x_test)
# #                     imp = model.feature_importances_
# #                     score = model.score(x_test,y_test)
# #                     print(score)
# #                     results['GBC'][str(ls)][str(lr)][str(est)][str(feat)][str(dep)]["score"] = score

# #                     f = pd.DataFrame(np.column_stack((features,imp)))
# #                     f.columns = ['feature','importance']
# #                     print(f.sort_values(by=['importance'],ascending=False))
# #                     results['GBC'][str(ls)][str(lr)][str(est)][str(feat)][str(dep)]["f_imp"] = f



In [113]:
# # tree models tabulation
# columns = ["model type","criterion","learning rate","max features","max depth","# of estimators","score"]
# l = []
# for mdl in results.keys():
#     if(mdl != "GBC"):
#         for crt in results[mdl].keys():
#             for mfeat in results[mdl][crt].keys():
#                 for mdep in results[mdl][crt][mfeat].keys():
#                     for nest in results[mdl][crt][mfeat][mdep].keys():
#                         score = results[mdl][crt][mfeat][mdep][nest]["score"]
#                         l.append([mdl,crt,None,mfeat,mdep,nest,score])
#     else:
#         for crt in results[mdl].keys():
#             for lr in results[mdl][crt].keys():
#                 for nest in results[mdl][crt][lr].keys():
#                     for mfeat in results[mdl][crt][lr][nest].keys():
#                         for mdep in results[mdl][crt][lr][nest][mfeat].keys():
#                             score = results[mdl][crt][lr][nest][mfeat][mdep]["score"]
#                             l.append([mdl,crt,lr,mfeat,mdep,nest,score])

# res = pd.DataFrame(l,columns=columns)
# res = res.sort_values(by=['score'],ascending=False).reset_index(drop=True)

# res.to_csv("ETree_Hyper.csv",sep=',',index=None)

# rs=pd.read_csv("ETree_Hyper.csv")

# rs

In [116]:
rs

,model type,criterion,learning rate,max features,max depth,# of estimators,score
0,ETC,gini,NaN,7,log2,200,0.683685
1,ETC,entropy,NaN,7,log2,200,0.683229
2,ETC,gini,NaN,7,sqrt,200,0.683223
3,ETC,entropy,NaN,7,sqrt,200,0.683210
4,ETC,gini,NaN,7,sqrt,120,0.682365
5,ETC,entropy,NaN,7,sqrt,120,0.682225
6,ETC,entropy,NaN,7,log2,120,0.682127
7,ETC,gini,NaN,7,log2,120,0.681966
8,ETC,gini,NaN,7,log2,50,0.678873
9,ETC,entropy,NaN,7,log2,50,0.678568


In [114]:
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.svm import SVC
# from sklearn.grid_search import GridSearchCV
# from sklearn.metrics import f1_score

# model_to_set = OneVsRestClassifier(SVC(kernel="poly"))

# parameters = {
#     "estimator__C": [1,2,4,8],
#     "estimator__kernel": ["poly","rbf"],
#     "estimator__degree":[1, 2, 3, 4],
# }

# model_tunning = GridSearchCV(model_to_set, param_grid=parameters,
#                              score_func=f1_score)

# model_tunning.fit(x_train,y_train)

# print (model_tunning.best_score_)
# print (model_tunning.best_params_)

In [115]:
# from sklearn import datasets
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.svm import SVC
# model1 = OneVsRestClassifier(SVC(kernel = 'poly')).fit(x_train, y_train)
# y_pred = model1.predict(x_test)
# #imp = model1.feature_importances_
# score = model1.score(x_test,y_test)
# print(score)

# #pd.DataFrame(np.column_stack((features,imp)))

In [117]:
#model type,criterion,learning rate,max features,max depth,# of estimators,score
#GBC,exponential,0.1,log2,7,100,0.900060606060606
#RFC,gini,,10,auto,80,0.6947878787878788
cr = "gini"
ls = 'exponential'
lr = 0.1
est = 100
feat = "log2"
dep = 7

In [118]:
RF = RandomForestClassifier(n_estimators=est,criterion=cr,max_features=feat,max_depth=dep,verbose=3)#,class_weight="balanced_subsample")
RF.fit(x_train,y_train)

y_pred = RF.predict(x_test)
imp = RF.feature_importances_
score = RF.score(x_test,y_test)
print(score)

pd.DataFrame(np.column_stack((features,imp)))

building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s


building tree 2 of 100


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.9s remaining:    0.0s


building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.4min finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   16.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


0.5747020743510499


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   15.7s finished


,0,1
0,Cosine,0.039076942139835064
1,Jaccard,0.07726737852295316
2,BM25,0.03786488545134396
3,Tfidf,0.0036924020853997936
4,Rouge P,0.32572457622177126
5,Rouge R,0.07073764665963952
6,BLEU 1,0.01035201997671064
7,BLEU 2,0.008352925429210999
8,DSSM,0.17154048806411593
9,CDSSM,0.21725754219180346


In [28]:



GB = GradientBoostingClassifier(loss=ls, learning_rate=lr,n_estimators=est,max_features=feat,max_depth=dep,verbose=3)

GB.fit(x_train,y_train)

y_pred = GB.predict(x_test)
imp = GB.feature_importances_
score = GB.score(x_test,y_test)
print(score)

pd.DataFrame(np.column_stack((features,imp)))

      Iter       Train Loss   Remaining Time 
         1           0.9328           27.77m
         2           0.9247           28.05m
         3           0.9175           27.01m
         4           0.9106           27.55m
         5           0.9049           27.03m
         6           0.8999           26.94m
         7           0.8958           26.58m
         8           0.8923           26.12m
         9           0.8893           24.81m
        10           0.8866           23.44m
        11           0.8843           22.14m
        12           0.8823           20.92m
        13           0.8806           19.88m
        14           0.8791           18.69m
        15           0.8778           18.69m
        16           0.8767           18.73m
        17           0.8758           18.78m
        18           0.8748           18.76m
        19           0.8740           18.76m
        20           0.8734           18.90m
        21           0.8728           18.85m
        2

,0,1
0,Cosine,0.05783912763326635
1,Jaccard,0.058568248858870124
2,BM25,0.09650918205001954
3,Tfidf,0.06301857709699635
4,Rouge P,0.09728945162175587
5,Rouge R,0.05300642988405758
6,BLEU 1,0.028148281062211403
7,BLEU 2,0.027359404471580674
8,DSSM,0.09795387735263995
9,CDSSM,0.10538927571037605


In [17]:
from sklearn.multiclass import OneVsRestClassifier
from pylightgbm.models import GBMRegressor
from sklearn.ensemble import ExtraTreesClassifier

ET = ExtraTreesClassifier(n_estimators=450, criterion='gini', verbose=0,n_jobs=-1, max_features='auto',min_samples_split=3, min_samples_leaf=20 )#, warm_start = True)


ET.fit(x_train,y_train)

y_pred = ET.predict(x_test)
imp = ET.feature_importances_
score = ET.score(x_test,y_test)
print(score)

pd.DataFrame(np.column_stack((features,imp)))

0.8464757653223447


,0,1
0,Cosine,0.05604296887956996
1,Jaccard,0.05472447135990491
2,BM25,0.05483435007815442
3,Tfidf,0.022055150947591015
4,Rouge P,0.3500241074594282
5,Rouge R,0.04762252031700716
6,BLEU 1,0.01034909511921164
7,BLEU 2,0.01044784907797101
8,DSSM,0.1263283708406751
9,CDSSM,0.14885650057407168


In [ ]:
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF

# kernel = 1.0 * RBF(1.0)
# gpc = GaussianProcessClassifier(kernel=kernel,
#         random_state=0).fit(x_train, y_train)

# score = gpc.score(x_test, y_test) 
# print(score)

# y_pred = ET.predict(x_test)
# imp = ET.feature_importances_

# pd.DataFrame(np.column_stack((features,imp)))

### Max Voting

In [35]:
from sklearn.ensemble import VotingClassifier
model = VotingClassifier(estimators=[('gradientboosting', GB), ('extratrees', ET)], voting='soft')
model.fit(x_train,y_train)
model.score(x_test,y_test)

      Iter       Train Loss   Remaining Time 
         1           0.9332           26.91m
         2           0.9249           28.00m
         3           0.9172           27.38m
         4           0.9107           27.03m
         5           0.9054           26.72m
         6           0.9006           26.70m
         7           0.8966           26.18m
         8           0.8931           25.79m
         9           0.8901           25.44m
        10           0.8876           25.83m
        11           0.8852           25.47m
        12           0.8830           25.03m
        13           0.8813           24.74m
        15           0.8784           24.13m
        16           0.8772           23.79m
        17           0.8761           23.51m
        18           0.8751           23.18m
        19           0.8743           22.96m
        20           0.8736           22.71m
        21           0.8729           22.39m
        22           0.8723           22.07m
        2

/opt/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6888360843732314

### Stacking

In [44]:
from sklearn.model_selection import StratifiedKFold

def Stacking(model,train,y,test,n_fold):
        folds=StratifiedKFold(n_splits=n_fold,random_state=1)
        test_pred=np.empty((test.shape[0],1),float)
        train_pred=np.empty((0,1),float)
        for train_indices,val_indices in folds.split(train,y.values):
            x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
            y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

            model.fit(X=x_train,y=y_train)
            train_pred=np.append(train_pred,model.predict(x_val))
            test_pred=np.append(test_pred,model.predict(test))
        return test_pred.reshape(-1,1),train_pred


In [45]:
model1 = ET

test_pred1 ,train_pred1=Stacking(model=model1,n_fold=10, train=x_train,test=x_test,y=y_train)

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [ ]:
model2 = GB

test_pred2 ,train_pred2=Stacking(model=model2,n_fold=10,train=x_train,test=x_test,y=y_train)

train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)

In [ ]:
df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)

model = LogisticRegression(random_state=1)
model.fit(df,y_train)
model.score(df_test, y_test)

### Blending


In [ ]:
# model1 = ET
# val_pred1=model1.predict(x_val)
# test_pred1=model1.predict(x_test)
# val_pred1=pd.DataFrame(val_pred1)
# test_pred1=pd.DataFrame(test_pred1)

# model2=GB
# val_pred2=model2.predict(x_val)
# test_pred2=model2.predict(x_test)
# val_pred2=pd.DataFrame(val_pred2)
# test_pred2=pd.DataFrame(test_pred2)

In [ ]:
# df_val=pd.concat([x_val, val_pred1,val_pred2],axis=1)
# df_test=pd.concat([x_test, test_pred1,test_pred2],axis=1)

# model = LogisticRegression()
# model.fit(df_val,y_val)
# model.score(df_test,y_test)

# Neural Network

In [87]:
inp = L.Input(shape=(len(features),))
x = L.Dense(128, activation='sigmoid')(inp)
x = L.Dense(256, activation='sigmoid')(x)
x = L.Dense(64, activation='sigmoid')(x)
out = L.Dense(1, activation='sigmoid')(x)

model = M.Model(inputs=[inp],outputs=[out])
model.summary()
 
o = Adamax(0.1)
model.compile(optimizer=o,loss='binary_crossentropy',metrics=['accuracy'])

model.fit(x_train,y_train,epochs=5,batch_size=128,validation_data=(x_test,y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 16)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               2176      
_________________________________________________________________
dense_14 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_15 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 65        
Total params: 51,713
Trainable params: 51,713
Non-trainable params: 0
_________________________________________________________________
Train on 733846 samples, validate on 1572540 samples
Epoch 1/5
733846/733846 [==============================] - 48s 66us/step - loss: 0.70

In [130]:
nntrain_raw = pd.read_csv("../Clean_data_latest.csv")
len(nntrain_raw)

5241770

In [131]:
nntrain = nntrain_raw[:1000].copy(deep=True)
nntrain.head()

,queryID,query,response,label,responseID
0,131,. corporation ?,"company incorporated specific nation , often w...",0,0
1,131,. corporation ?,"today , growing community 2,100 certified b co...",0,1
2,131,. corporation ?,"corporation definition , association individua...",0,2
3,131,. corporation ?,example corporation sentence . 1 work consulta...,0,3
4,131,. corporation ?,1 : government-owned corporation ( utility rai...,0,4


In [ ]:
def clean(df):
    
  column = 'query'
  
  #Split a-b into a and b
  df[column] = df[column].str.replace('-',' ')
  print(column+": - Replaced")
  #Removing punctuations
  df[column] = df[column].str.replace('[^\w\s]','')
  print(column+": Removed punctions ")
  #Replacing numbers
  df[column] = df[column].str.replace('[0-9]+','#')
  print(column+": Replaced Numbers ")
  #Stemming
  #df[column] = df[column].apply(lambda x: " ".join([porter.stem(word) for word in x.split()]))
  #print(column+": Stemming done")
  
  column = 'response'
  #Split a-b into a and b
  df[column] = df[column].str.replace('-',' ')
  print(column+": - Replaced")
  #Removing punctuations
  df[column] = df[column].str.replace('[^\w\s]','')
  print(column+": Removed punctions ")
  #Replacing numbers
  df[column] = df[column].str.replace('[0-9]+','#')
  print(column+": Replaced Numbers ")
  #Stemming
  #df[column] = df[column].apply(lambda x: " ".join([porter.stem(word) for word in x.split()]))
  #print(column+": Stemming done")
  
  return df
  

# MRR

In [93]:
test_labelID = pd.read_csv("Train_labelID.csv",sep=',',header=None).iloc[split:]
test_queryID = pd.read_csv("Train_QueryID.csv",sep=',',header=None).iloc[split:]

In [119]:
#if NN
#probs = model.predict(x_test).ravel()
#else
probs = RF.predict_proba(x_test)[:,1]

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   15.7s finished


In [120]:
p = []
for i in range(0,probs.shape[0],10):
  l = []
  v = probs[i:i+10].T
  l.append(test_queryID.values[i][0])
  l.extend(v)
  p.append(l)

p = np.array(p)
y_pred = pd.DataFrame(p[:,1:],index=p[:,0].astype(int))

In [121]:
tmp=pd.DataFrame(test_labelID.values.ravel(),index=test_queryID.astype(int).values.ravel()).reset_index(drop=True)
h = []
for i in range(0,tmp.shape[0],10):
  h.append(tmp.values[i])
y_true = pd.DataFrame(h)

In [122]:
y_pred.columns = list(range(10))
y_true.columns = ['labelID']

In [140]:
def MRR(sub,ref):
      scores = []
      for q_id in ref.index:
        position = ref.iloc[q_id].values.ravel()
        print(position)
        rank = np.argsort(sub.iloc[q_id])[::-1].tolist().index(position)
        rank +=1
        scores.append(1.0/rank)
        print(rank)
      score = np.mean(scores)
      print(score)

In [147]:
# y_pred[:2]

In [148]:
# y_true[:2]

In [149]:
MRR(y_pred,y_true)

# MRR2

In [18]:
test_labelID = pd.read_csv("Train_labelID.csv",sep=',').iloc[split:]
test_queryID = pd.read_csv("Train_queryID.csv",sep=',').iloc[split:]#pd.read_csv("Train_QueryID.csv",sep=',',header=None).iloc[split:]

In [19]:
#if NN
#probs = model.predict(x_test).ravel()
#else
probs = ET.predict_proba(x_test)[:,1]

In [20]:
def get_MRR_df(probs,test_queryID,test_labelID):
  p = []
  q = []
  for i in range(0,probs.shape[0],10):
    l = []
    h = []    
    l.append(int(test_queryID[i]))
    h.append(int(test_queryID.ravel()[i]))
    
    v = probs[i:i+10].T
    l.extend(v)
    h.append(test_labelID.ravel()[i])
    
    p.append(l)
    q.append(h)

  y_pred = pd.DataFrame(p)
  y_pred.columns = ['queryID']+list(range(10))
  y_true = pd.DataFrame(q)
  y_true.columns = ['queryID','labelID']
  y_true = y_true.sort_values(by=['queryID']).reset_index(drop=True)
  y_pred = y_pred.sort_values(by=['queryID']).reset_index(drop=True)
  return y_pred, y_true

In [21]:
y_pred, y_true = get_MRR_df(probs,test_queryID.values,test_labelID.values)

In [22]:
def MRR(sub,ref):
  scores = []
  for i in range(ref.shape[0]):
    position = ref['labelID'][i]
#     print(position)
    rank = np.argsort(sub.iloc[i])[::-1].tolist().index(position)
#     print(rank)
    rank +=1
    scores.append(1.0/rank)

  score = np.mean(scores)
  print(score)

In [23]:
MRR(y_pred,y_true)

0.3564009179384623


# Evaluation Code

In [125]:
d2 = pd.read_csv("All_TestFeatures.csv",sep=',')

x_test_actual = d2[features]

In [38]:
#X_test = x_test_actual.fillna(x_train.mean())
#d2.head(20)

In [89]:
# '''
# IF Neural Net Model

# '''
# probs = model.predict(x_test_actual).ravel()

# p = []
# for i in range(0,d2.shape[0],10):
#     l = []
#     v = probs[i:i+10]
#     l.append(d2['qid'][i])
#     l.extend(v)
#     p.append(l)

# result = pd.DataFrame(p)

# result.to_csv("answer.tsv",sep='\t',header=None,index=False)

In [127]:
'''
IF Tree Model

'''
probs = RF.predict_proba(x_test_actual)[:,1]

p = []
for i in range(0,d2.shape[0],10):
    l = []
    v = probs[i:i+10].T
    l.append(d2['qid'][i])
    l.extend(v)
    p.append(l)

result = pd.DataFrame(p)

result.to_csv("answer.tsv",sep='\t',header=None,index=False)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.2s finished


In [41]:
# x_test_actual.head(30)

In [42]:
# x_test_new = pd.concat([x_test_actual,  pd.DataFrame({'DSSM':0, 'CDSSM':0}, index=[0]), x_test_actual.loc[1:]], ignore_index=True).copy(deep=True)
# x_test_new.head()